In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
import matplotlib
%matplotlib inline

In [2]:
iris = datasets.load_iris()
df = pd.DataFrame(data=iris['data'], columns=iris['feature_names'])
i = pd.Series(iris['target'])
names = i.map(lambda x: iris['target_names'][x])
df['index'] = i
df['category_name'] = names
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),index,category_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
5,5.4,3.9,1.7,0.4,0,setosa
6,4.6,3.4,1.4,0.3,0,setosa
7,5.0,3.4,1.5,0.2,0,setosa
8,4.4,2.9,1.4,0.2,0,setosa
9,4.9,3.1,1.5,0.1,0,setosa


In [87]:
import edward as ed
import tensorflow as tf
from edward.models import Bernoulli, MultivariateNormalTriL, Normal
from edward.util import rbf

# shuffle
df = df.sample(frac=1)

ys = df['index'].values
xs = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']].values
x_train = xs[:100]
y_train = ys[:100]
x_test = xs[100:]
y_test = ys[100:]
print("Number of train data: {}".format(x_train.shape[0]))
print("Number of features: {}".format(x_train.shape[1]))

Number of train data: 100
Number of features: 4


In [88]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder

In [89]:
encoder = OneHotEncoder(sparse=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.fit_transform(y_test.reshape(-1, 1))


model = Sequential()

model.add(Dense(10, input_shape=(4,), activation='relu', name='fc1'))
model.add(Dense(10, activation='relu', name='fc2'))
model.add(Dense(3, activation='softmax', name='output'))

optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('Neural Network Model Summary: ')
print(model.summary())

model.fit(x_train, y_train, verbose=2, batch_size=5, epochs=30)

results = model.evaluate(x_test, y_test)
results

Neural Network Model Summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
fc1 (Dense)                  (None, 10)                50        
_________________________________________________________________
fc2 (Dense)                  (None, 10)                110       
_________________________________________________________________
output (Dense)               (None, 3)                 33        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
 - 0s - loss: 0.9998 - acc: 0.6100
Epoch 2/30
 - 0s - loss: 0.8910 - acc: 0.7100
Epoch 3/30
 - 0s - loss: 0.8243 - acc: 0.7400
Epoch 4/30
 - 0s - loss: 0.7752 - acc: 0.8200
Epoch 5/30
 - 0s - loss: 0.7337 - acc: 0.8000
Epoch 6/30
 - 0s - loss: 0.6982 - acc: 0.8300
Epoch 7/30
 - 0s - loss: 0.6610 - acc: 0.8300
Epoch 8/30
 - 0s - loss: 0.6233 - acc: 0.8500
Epoc

[0.2968025326728821, 0.94]